In [ ]:
# Importing the libraries 

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import plot_roc_curve
from sklearn.metrics import accuracy_score, classification_report

In [ ]:
# Importing dataset
from google.colab import files
uploaded = files.upload()

Saving GOOGL.csv to GOOGL.csv


In [ ]:
# Load the data
df = pd.read_csv('GOOGL.csv')

# Display the head before moving on.
df.head() 

,Date,Open,High,Low,Close,Adj Close,Volume
0,2018-01-02,1053.020020,1075.979980,1053.020020,1073.209961,1073.209961,1588300
1,2018-01-03,1073.930054,1096.099976,1073.430054,1091.520020,1091.520020,1565900
2,2018-01-04,1097.089966,1104.079956,1094.260010,1095.760010,1095.760010,1302600
3,2018-01-05,1103.449951,1113.579956,1101.800049,1110.290039,1110.290039,1512500
4,2018-01-08,1111.000000,1119.160034,1110.000000,1114.209961,1114.209961,1232200


In [ ]:
df.tail()

,Date,Open,High,Low,Close,Adj Close,Volume
246,2018-12-24,984.320007,1012.119995,977.659973,984.669983,984.669983,1818000
247,2018-12-26,997.989990,1048.449951,992.650024,1047.849976,1047.849976,2315900
248,2018-12-27,1026.199951,1053.339966,1007.000000,1052.900024,1052.900024,2299800
249,2018-12-28,1059.500000,1064.229980,1042.000000,1046.680054,1046.680054,1719900
250,2018-12-31,1057.829956,1062.989990,1033.040039,1044.959961,1044.959961,1655500


In [ ]:
# calculate lag 1
df['Lag1'] = df['Adj Close'].diff(1)

# calculate lag 2
df['Lag2'] = df['Adj Close'].diff(2)

# calculate Lag 3
df['Lag3'] = df['Adj Close'].diff(3)

# calculate lag 4
df['Lag4'] = df['Adj Close'].diff(4)

# calculate lag 5
df['Lag5'] = df['Adj Close'].diff(5)

# Creating percantage change column 

df['% Change'] = df['Adj Close'].pct_change()


df.head(10)

,Date,Open,High,Low,Close,Adj Close,Volume,Lag1,Lag2,Lag3,Lag4,Lag5,% Change
0,2018-01-02,1053.020020,1075.979980,1053.020020,1073.209961,1073.209961,1588300,NaN,NaN,NaN,NaN,NaN,NaN
1,2018-01-03,1073.930054,1096.099976,1073.430054,1091.520020,1091.520020,1565900,18.310059,NaN,NaN,NaN,NaN,0.017061
2,2018-01-04,1097.089966,1104.079956,1094.260010,1095.760010,1095.760010,1302600,4.239990,22.550049,NaN,NaN,NaN,0.003884
3,2018-01-05,1103.449951,1113.579956,1101.800049,1110.290039,1110.290039,1512500,14.530029,18.770019,37.080078,NaN,NaN,0.013260
4,2018-01-08,1111.000000,1119.160034,1110.000000,1114.209961,1114.209961,1232200,3.919922,18.449951,22.689941,41.000000,NaN,0.003531
5,2018-01-09,1118.439941,1118.439941,1108.199951,1112.790039,1112.790039,1340400,-1.419922,2.500000,17.030029,21.270019,39.580078,-0.001274
6,2018-01-10,1107.000000,1112.780029,1103.979980,1110.140015,1110.140015,1036700,-2.650024,-4.069946,-0.150024,14.380005,18.619995,-0.002381
7,2018-01-11,1112.310059,1114.849976,1106.479980,1112.050049,1112.050049,1121200,1.910034,-0.739990,-2.159912,1.760010,16.290039,0.001721
8,2018-01-12,1110.099976,1131.300049,1108.010010,1130.650024,1130.650024,1929300,18.599975,20.510009,17.859985,16.440063,20.359985,0.016726
9,2018-01-16,1140.310059,1148.880005,1126.660034,1130.699951,1130.699951,1823100,0.049927,18.649902,20.559936,17.909912,16.489990,0.000044


In [ ]:
#Creating directional change 
close_groups = df['Adj Close']

# Apply the lambda function which will return -1.0 for down, 1.0 for up and 0.0 for no change.
close_groups = close_groups.transform(lambda x : np.sign(x.diff()))

# add the data to the main dataframe.
df['Prediction'] = close_groups

# print the head
df.head(10)

,Date,Open,High,Low,Close,Adj Close,Volume,Lag1,Lag2,Lag3,Lag4,Lag5,% Change,Prediction
0,2018-01-02,1053.020020,1075.979980,1053.020020,1073.209961,1073.209961,1588300,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2018-01-03,1073.930054,1096.099976,1073.430054,1091.520020,1091.520020,1565900,18.310059,NaN,NaN,NaN,NaN,0.017061,1.0
2,2018-01-04,1097.089966,1104.079956,1094.260010,1095.760010,1095.760010,1302600,4.239990,22.550049,NaN,NaN,NaN,0.003884,1.0
3,2018-01-05,1103.449951,1113.579956,1101.800049,1110.290039,1110.290039,1512500,14.530029,18.770019,37.080078,NaN,NaN,0.013260,1.0
4,2018-01-08,1111.000000,1119.160034,1110.000000,1114.209961,1114.209961,1232200,3.919922,18.449951,22.689941,41.000000,NaN,0.003531,1.0
5,2018-01-09,1118.439941,1118.439941,1108.199951,1112.790039,1112.790039,1340400,-1.419922,2.500000,17.030029,21.270019,39.580078,-0.001274,-1.0
6,2018-01-10,1107.000000,1112.780029,1103.979980,1110.140015,1110.140015,1036700,-2.650024,-4.069946,-0.150024,14.380005,18.619995,-0.002381,-1.0
7,2018-01-11,1112.310059,1114.849976,1106.479980,1112.050049,1112.050049,1121200,1.910034,-0.739990,-2.159912,1.760010,16.290039,0.001721,1.0
8,2018-01-12,1110.099976,1131.300049,1108.010010,1130.650024,1130.650024,1929300,18.599975,20.510009,17.859985,16.440063,20.359985,0.016726,1.0
9,2018-01-16,1140.310059,1148.880005,1126.660034,1130.699951,1130.699951,1823100,0.049927,18.649902,20.559936,17.909912,16.489990,0.000044,1.0


In [ ]:
# Any row that has a `NaN` value will be dropped.
df = df.dropna()

# Print the head.
df.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Lag1,Lag2,Lag3,Lag4,Lag5,% Change,Prediction
5,2018-01-09,1118.439941,1118.439941,1108.199951,1112.790039,1112.790039,1340400,-1.419922,2.500000,17.030029,21.270019,39.580078,-0.001274,-1.0
6,2018-01-10,1107.000000,1112.780029,1103.979980,1110.140015,1110.140015,1036700,-2.650024,-4.069946,-0.150024,14.380005,18.619995,-0.002381,-1.0
7,2018-01-11,1112.310059,1114.849976,1106.479980,1112.050049,1112.050049,1121200,1.910034,-0.739990,-2.159912,1.760010,16.290039,0.001721,1.0
8,2018-01-12,1110.099976,1131.300049,1108.010010,1130.650024,1130.650024,1929300,18.599975,20.510009,17.859985,16.440063,20.359985,0.016726,1.0
9,2018-01-16,1140.310059,1148.880005,1126.660034,1130.699951,1130.699951,1823100,0.049927,18.649902,20.559936,17.909912,16.489990,0.000044,1.0


In [ ]:
# Preparing the training data: 1/01/2018 -11/30/2018
df_train =df.iloc[:226].dropna().copy()
df_train


,Date,Open,High,Low,Close,Adj Close,Volume,Lag1,Lag2,Lag3,Lag4,Lag5,% Change,Prediction
5,2018-01-09,1118.439941,1118.439941,1108.199951,1112.790039,1112.790039,1340400,-1.419922,2.500000,17.030029,21.270019,39.580078,-0.001274,-1.0
6,2018-01-10,1107.000000,1112.780029,1103.979980,1110.140015,1110.140015,1036700,-2.650024,-4.069946,-0.150024,14.380005,18.619995,-0.002381,-1.0
7,2018-01-11,1112.310059,1114.849976,1106.479980,1112.050049,1112.050049,1121200,1.910034,-0.739990,-2.159912,1.760010,16.290039,0.001721,1.0
8,2018-01-12,1110.099976,1131.300049,1108.010010,1130.650024,1130.650024,1929300,18.599975,20.510009,17.859985,16.440063,20.359985,0.016726,1.0
9,2018-01-16,1140.310059,1148.880005,1126.660034,1130.699951,1130.699951,1823100,0.049927,18.649902,20.559936,17.909912,16.489990,0.000044,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
226,2018-11-23,1033.500000,1043.000000,1028.520020,1030.099976,1030.099976,708900,-13.330078,-0.349975,2.679932,-38.170044,-40.950073,-0.012775,-1.0
227,2018-11-26,1044.000000,1057.000000,1039.770020,1055.939941,1055.939941,1577900,25.839965,12.509887,25.489990,28.519897,-12.330079,0.025085,1.0
228,2018-11-27,1051.339966,1066.760010,1045.500000,1052.280029,1052.280029,1652400,-3.659912,22.180053,8.849975,21.830078,24.859985,-0.003466,-1.0
229,2018-11-28,1057.459961,1092.859985,1042.949951,1091.790039,1091.790039,2302100,39.510010,35.850098,61.690063,48.359985,61.340088,0.037547,1.0


In [ ]:
# Creating testing data 

df_test = df.iloc[226:].dropna().copy()
df_test

,Date,Open,High,Low,Close,Adj Close,Volume,Lag1,Lag2,Lag3,Lag4,Lag5,% Change,Prediction
231,2018-11-30,1095.630005,1109.650024,1083.180054,1109.650024,1109.650024,2983400,15.070068,17.859985,57.369995,53.710083,79.550048,0.013768,1.0
232,2018-12-03,1132.160034,1135.000000,1111.729980,1116.359985,1116.359985,2876600,6.709961,21.780029,24.569946,64.079956,60.420044,0.006047,1.0
233,2018-12-04,1112.989990,1114.239990,1060.780029,1062.469971,1062.469971,2281500,-53.890014,-47.180053,-32.109985,-29.320068,10.189942,-0.048273,-1.0
234,2018-12-06,1045.000000,1079.420044,1042.479980,1078.079956,1078.079956,2371800,15.609985,-38.280029,-31.570068,-16.500000,-13.710083,0.014692,1.0
235,2018-12-07,1072.229980,1085.030029,1039.369995,1046.579956,1046.579956,2134100,-31.500000,-15.890015,-69.780029,-63.070068,-48.000000,-0.029219,-1.0
236,2018-12-10,1042.939941,1059.599976,1033.000000,1053.180054,1053.180054,1682600,6.600098,-24.899902,-9.289917,-63.179931,-56.469970,0.006306,1.0
237,2018-12-11,1066.939941,1070.400024,1050.099976,1061.650024,1061.650024,1692600,8.469970,15.070068,-16.429932,-0.819947,-54.709961,0.008042,1.0
238,2018-12-12,1077.079956,1091.729980,1071.969971,1073.729980,1073.729980,1448000,12.079956,20.549926,27.150024,-4.349976,11.260009,0.011378,1.0
239,2018-12-13,1075.670044,1088.420044,1064.989990,1073.540039,1073.540039,1249300,-0.189941,11.890015,20.359985,26.960083,-4.539917,-0.000177,-1.0
240,2018-12-14,1060.020020,1071.719971,1049.369995,1051.709961,1051.709961,1817900,-21.830078,-22.020019,-9.940063,-1.470093,5.130005,-0.020335,-1.0


In [ ]:
# Creating X and Y columns 

X_train = df_train[['Lag1','Lag2','Lag3','Lag4','Lag5']] 
y_train = df_train['Prediction']
X_test = df_test[['Lag1','Lag2','Lag3','Lag4','Lag5']] 
y_test = df_test['Prediction']


In [ ]:
# Create a Random Forest Classifier
rand_frst_clf = RandomForestClassifier(n_estimators = 100)

In [ ]:
# Fit the data to the model
rand_frst_clf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [ ]:
# Make predictions
y_pred = rand_frst_clf.predict(X_test)
y_pred

array([ 1.,  1., -1.,  1., -1.,  1.,  1.,  1., -1., -1., -1.,  1., -1.,
       -1., -1., -1.,  1.,  1., -1., -1.])

In [ ]:
# Print the Accuracy of our Model.
print('Correct Prediction (%): ', accuracy_score(y_test, y_pred, normalize = True) * 100.0)

Correct Prediction (%):  100.0
